In [1]:
import os
import io
import json
import openai
import requests
import tiktoken
from objprint import op
from chatgpter import CallChatGPT, password_generator
from IPython.display import Markdown

In [2]:
with open("api_key.txt", "r") as f:
    api_key = f.read().split("\n")
    api_key = api_key[0]
    
input_text = "这是一次测试！"

# 测试API

## openai.ChatCompletion.create stream=False

In [ ]:
openai.api_key = api_key
messages = [{"role": "user", "content": input_text}]
prompt = "You are a helpful assistant."
messages.insert(0, {"role": "system", "content": prompt})
prompt = "You are a helpful assistant."
messages.insert(0, {"role": "system", "content": prompt})
response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                        messages=messages,
                                        n=1,
                                        stream=False)

print(response)
print(response["usage"])
print(response["choices"][0]["message"]["content"])

## openai.ChatCompletion.create stream=True

In [ ]:
openai.api_key = api_key
messages = [{"role": "user", "content": input_text}]
prompt = "You are a helpful assistant."
messages.insert(0, {"role": "system", "content": prompt})
response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                        messages=messages,
                                        stream=True,)

string_io = io.StringIO()
for idx, chunk in enumerate(response):
    print(chunk)
    if chunk["choices"][0]["finish_reason"] != "stop": 
        if chunk["choices"][0].get("delta"):
            if chunk["choices"][0]["delta"].get("content"):
                string_io.write(chunk["choices"][0]["delta"]["content"])
                print(string_io.getvalue())

print(idx, string_io.getvalue())

## openai.ChatCompletion.create stream=True gradio

In [ ]:
def predict(messages):
    openai.api_key = api_key
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                            messages=messages,
                                            stream=True,)

    return response


def get_stream_value(input_text):
    output_text = ""
    string_io = io.StringIO()
    yield string_io.getvalue()
    
    messages = [{"role": "user", "content": f"{input_text}"}]
    prompt = "You are a helpful assistant."
    messages.insert(0, {"role": "system", "content": prompt})

    response = predict(messages)
    
    messages.append({})
    for idx, chunk in enumerate(response):
        if chunk["choices"][0]["finish_reason"] != "stop":
            if chunk["choices"][0].get("delta"):
                if chunk["choices"][0]["delta"].get("content"):
                    string_io.write(chunk["choices"][0]["delta"]["content"])
                    output_text = string_io.getvalue()
                    messages[-1] = {"role": "assistant", "content": output_text}
                    yield string_io.getvalue()
    
    print(idx, output_text, messages)
     

with gr.Blocks() as demo:
    with gr.Row(variant="default").style(equal_height=True):
        input_text = gr.Textbox(label="in")
        output_text = gr.Textbox(label="out")
    input_text.submit(fn=get_stream_value, inputs=[input_text], outputs=[output_text])
    

demo.queue().launch()

## requres.post stream=False

In [ ]:
messages = [{"role": "user", "content": input_text}] 
prompt = "You are a helpful assistant."
messages.insert(0, {"role": "system", "content": prompt})
payload = {
    "model": "gpt-3.5-turbo",
    "messages": messages,
    "temperature": 1,
    "top_p": 1,
    "n": 1,
    "stream": False,
    "presence_penalty": 0,
    "frequency_penalty": 0,
}
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
proxies = {
    "http": f"http://127.0.0.1:{7890}",
    "https": f"http://127.0.0.1:{7890}"
}
api_url = "https://api.openai.com/v1/chat/completions"

response = requests.post(api_url,
                        json=payload,
                        headers=headers,
                        timeout=30,
                        proxies=proxies,
                        stream=False,)

response = json.loads(response.text)
op(response)
op(response["usage"])
print(response["choices"][0]["message"]["content"])

## requres.post stream=True

In [ ]:
try:
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
except KeyError:
    tokenizer = tiktoken.get_encoding("cl100k_base")
messages = [{"role": "user", "content": input_text}]
prompt = "You are a helpful assistant."
messages.insert(0, {"role": "system", "content": prompt})

message = "".join([f"role: {msg['role']}, content: {msg['content']}" for msg in messages])
print(len(tokenizer.encode(message)))

In [ ]:
## socket

In [ ]:
def is_port_in_use(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

print(is_port_in_use(7890))

# 测试模型

In [ ]:
gpt_model = CallChatGPT(model="gpt-3.5-turbo", n=1)
display(Markdown(input_text))
for answer in gpt_model(prompt=input_text):
    display(Markdown(answer))

In [ ]:
display(f"日志密码 {password_generator()}")
os.system(f"python -m chatgpter -ak {api_key}")